In [12]:
# add more info (e.g., cross references) (Timing: ~ 500 s)

# data stored in /supplData
# 'chem_xref.tsv' (https://www.metanetx.org/cgi-bin/mnxget/mnxref/chem_xref.tsv)
# 'reac_xref.tsv' (https://www.metanetx.org/cgi-bin/mnxget/mnxref/reac_xref.tsv)

import tarfile, re
import pandas as pd
import time
time_start = time.time()

keggdir = '..' # the folder of the downloaded KEGG database

def getDictMNX(MNX2db, line, db):
    dbtmp = [re.findall(db+':\S+', line)[0].replace(db+':','')]
    MNXtmp = re.findall('MNX[RM]\d+', line)
    if len(MNXtmp) > 0:
        if MNXtmp[0] in MNX2db:
            MNX2db[MNXtmp[0]] += dbtmp # are there multiple MNX IDs to one KEGG ID?
        else:
            MNX2db[MNXtmp[0]] = dbtmp
    return MNX2db

def convertDict(kegg2MNX,MNX2db):
    kegg2db = dict()
    for key in kegg2MNX:
        dbvalue = []
        for MNX in kegg2MNX[key]:
            if MNX in MNX2db:
                dbvalue += MNX2db[MNX]
        kegg2db[key] = dbvalue
    return kegg2db


# Compound info
# SMILES and images
import os
import math
import pickle
import numpy as np
from rdkit import Chem
import tarfile
import glob
from rdkit.Chem import Draw
from pathlib import Path
# generate images from mol file and output smiles
exceptionlist = list()
compound_dict = dict()
tar = tarfile.open(keggdir + '/kegg/ligand/compound.tar.gz', 'r:gz')
mollist = [i for i in tar.getnames() if 'compound/mol/' in i]

def dump_file(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)
        
for molfile in mollist:
    try:
        stringWithMolData = tar.extractfile(molfile).read().decode()
        molstruct = Chem.MolFromMolBlock(stringWithMolData)
        #print(molstruct)
        p = Path(molfile)
        compound = p.stem
        #print(compound)
        compound_dict[compound] = Chem.MolToSmiles(molstruct)
        #print(compound_dict[compound])
        #Draw.MolToFile(molsturct, 'images/' + compound + '.png',size=(400,400))
        img=Draw.MolsToGridImage({molstruct}, molsPerRow=1, subImgSize=(300, 160), useSVG=True)
        with open('images/' + compound + '.svg', 'w') as f_handle:
            f_handle.write(img.data)
    except:
        p = Path(molfile)
        compound = p.stem
        exceptionlist.append(compound)

dump_file(compound_dict, 'output/kegg_smiles_dict.pickle')


# KEGG data
tar = tarfile.open(keggdir + '/kegg/ligand/compound.tar.gz', 'r:gz')
f = tar.extractfile('compound/compound')
cmpd_name = dict() # compound name (only the first name is used when there are multiple names)
cmpd_formula = dict() # compound formula
for line in f:
    if line.decode().startswith('ENTRY'):
        c_id = re.findall('[A-Z]\d{5}', line.decode())[0]
        cmpd_name[c_id] = []
        cmpd_formula[c_id] = []
    elif line.decode().startswith('NAME'):
        cmpd_name[c_id] = [line.decode().replace('NAME','').strip().replace(';','')]
    elif line.decode().startswith('FORMULA'):
        cmpd_formula[c_id] = [line.decode().replace('FORMULA','').strip().replace(';','')]
f.close()
tar.close()
# Other sources
f = open('supplData/chem_xref.tsv','r')
keggC2MNXM = dict()
MNXM2seedM = dict()
MNXM2biggM = dict()
MNXM2chebi = dict()
MNXM2metacycM = dict()
MNXM2sabiorkM = dict()
MNXM2reactomeM = dict()
for line in f:
    if line.startswith('kegg') and line.find('secondary/obsolete/fantasy identifier') == -1:
        keggC = re.findall('[A-Z]\d{5}', line)[0]
        MNXM = re.findall('MNXM\d+', line)
        if keggC in keggC2MNXM:
            keggC2MNXM[keggC] += MNXM # are there multiple MNXM IDs to one KEGG ID?
            keggC2MNXM[keggC] = list(set(keggC2MNXM[keggC]))
        else:
            keggC2MNXM[keggC] = MNXM
    elif line.startswith('seedM') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2seedM = getDictMNX(MNXM2seedM, line, 'seedM')
    elif line.startswith('biggM') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2biggM = getDictMNX(MNXM2biggM, line, 'biggM')
    elif line.startswith('chebi') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2chebi = getDictMNX(MNXM2chebi, line, 'chebi')
    elif line.startswith('metacycM') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2metacycM = getDictMNX(MNXM2metacycM, line, 'metacycM')
    elif line.startswith('sabiorkM') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2sabiorkM = getDictMNX(MNXM2sabiorkM, line, 'sabiorkM')
    elif line.startswith('reactomeM') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXM2reactomeM = getDictMNX(MNXM2reactomeM, line, 'reactomeM')
keggC2seedM = convertDict(keggC2MNXM,MNXM2seedM)
keggC2biggM = convertDict(keggC2MNXM,MNXM2biggM)
keggC2chebi = convertDict(keggC2MNXM,MNXM2chebi)
keggC2metacycM = convertDict(keggC2MNXM,MNXM2metacycM)
keggC2sabiorkM = convertDict(keggC2MNXM,MNXM2sabiorkM)
keggC2reactomeM = convertDict(keggC2MNXM,MNXM2reactomeM)
# write file
fout = open('supplOutput/compound.txt', 'w')
fout.write('KEGG\t'+'Name\t'+'Formula\t'+'MetaNetX\t'+'ModelSEED\t'+'BiGG\t'+'ChEBI\t'+'MetaCyc\t'+'SABIO-RK\t'+'Reactome\t'+'SMILES'+'\n')
for key in cmpd_name:
    if key in compound_dict:
        smile = compound_dict[key]
    else:
        smile = ''
    if key in keggC2MNXM:
        line_to_write = key + '\t' + ';'.join(cmpd_name[key]) + '\t' + ';'.join(cmpd_formula[key]) + '\t' + ';'.join(keggC2MNXM[key]) + '\t'\
        + ';'.join(keggC2seedM[key]) + '\t' + ';'.join(keggC2biggM[key]) + '\t' + ';'.join(keggC2chebi[key]) + '\t'\
        + ';'.join(keggC2metacycM[key]) + '\t' + ';'.join(keggC2sabiorkM[key]) + '\t' + ';'.join(keggC2reactomeM[key]) + '\t' + smile + '\n'
    else:
        line_to_write = key + '\t' + ';'.join(cmpd_name[key]) + '\t' + ';'.join(cmpd_formula[key]) + '\t\t\t\t\t\t\t' + '\t' + smile + '\n'
    fout.write(line_to_write)
fout.close()


# Reaction info
# KEGG data
tar = tarfile.open(keggdir + '/kegg/ligand/reaction.tar.gz', 'r:gz')
f = tar.extractfile('reaction/reaction')
rxn_name = dict() # reaction name (only the first name is used when there are multiple names)
rxn_formula = dict() # reaction formula
for line in f:
    if line.decode().startswith('ENTRY'):
        r_id = re.findall('R\d{5}', line.decode())[0]
        rxn_name[r_id] = []
        rxn_formula[r_id] = []
    elif line.decode().startswith('NAME'):
        rxn_name[r_id] = [line.decode().replace('NAME','').strip().replace(';','')]
    elif line.decode().startswith('DEFINITION'):
        rxn_formula[r_id] = [line.decode().replace('DEFINITION','').strip().replace(';','')]
f.close()
tar.close()
# Other sources
f = open('supplData/reac_xref.tsv','r')
keggR2MNXR = dict()
MNXR2rheaR = dict()
MNXR2seedR = dict()
MNXR2biggR = dict()
MNXR2metacycR = dict()
MNXR2sabiorkR = dict()
for line in f:
    if line.startswith('keggR'):
        keggC = re.findall('keggR:R\d{5}', line)[0].replace('keggR:','')
        MNXM = re.findall('MNXR\d+', line)
        if keggC in keggR2MNXR: 
            keggR2MNXR[keggC] += MNXM # are there multiple MNXR IDs to one KEGG ID?
        else:
            keggR2MNXR[keggC] = MNXM
    elif line.startswith('rheaR') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXR2rheaR = getDictMNX(MNXR2rheaR, line, 'rheaR')
    elif line.startswith('seedR') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXR2seedR = getDictMNX(MNXR2seedR, line, 'seedR')
    elif line.startswith('biggR') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXR2biggR = getDictMNX(MNXR2biggR, line, 'biggR')
    elif line.startswith('metacycR') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXR2metacycR = getDictMNX(MNXR2metacycR, line, 'metacycR')
    elif line.startswith('sabiorkR') and line.find('secondary/obsolete/fantasy identifier') == -1:
        MNXR2sabiorkR = getDictMNX(MNXR2sabiorkR, line, 'sabiorkR')
keggR2rheaR = convertDict(keggR2MNXR,MNXR2rheaR)
keggR2seedR = convertDict(keggR2MNXR,MNXR2seedR)
keggR2biggR = convertDict(keggR2MNXR,MNXR2biggR)
keggR2metacycR = convertDict(keggR2MNXR,MNXR2metacycR)
keggR2sabiorkR = convertDict(keggR2MNXR,MNXR2sabiorkR)
# write file
fout = open('supplOutput/reaction.txt', 'w')
fout.write('KEGG\t'+'Name\t'+'Equation\t'+'MetaNetX\t'+'Rhea\t'+'ModelSEED\t'+'BiGG\t'+'MetaCyc\t'+'SABIO-RK'+'\n')
for key in rxn_name:
    if key in keggR2MNXR:
        line_to_write = key + '\t' + ';'.join(rxn_name[key]) + '\t' + ';'.join(rxn_formula[key]) + '\t' + ';'.join(keggR2MNXR[key]) + '\t'\
        + ';'.join(keggR2rheaR[key]) + '\t' + ';'.join(keggR2seedR[key]) + '\t' + ';'.join(keggR2biggR[key]) + '\t'\
        + ';'.join(keggR2metacycR[key]) + '\t' + ';'.join(keggR2sabiorkR[key]) + '\n'
    else:
        line_to_write = key + '\t' + ';'.join(rxn_name[key]) + '\t' + ';'.join(rxn_formula[key]) + '\t\t\t\t\t\t\n'
    fout.write(line_to_write)
fout.close()


# Enzyme info
fout = open('supplOutput/ec.txt', 'w')
fout.write('EC' + '\t' + 'Name' + '\n')
# KEGG data
tar = tarfile.open(keggdir + '/kegg/ligand/enzyme.tar.gz', 'r:gz')
f = tar.extractfile('enzyme/enzyme')
ec_name = dict() # EC number name
flag = False # used for names in multiple lines
for line in f:
    if line.decode().startswith('ENTRY'):
        ec = re.findall('\S+\.\S+\.\S+\.\S+', line.decode())[0]
        ec_name[ec] = []
    elif line.decode().startswith('NAME'):
        ec_name[ec] = line.decode().replace('NAME','').strip().replace(';','')
        if line.decode().endswith(';\n'):
            flag = True
    # for names in multiple lines
    elif flag and line.decode().startswith(' '):
        ec_name[ec] += ';' + line.decode().strip().replace(';','')
        if line.decode().endswith(';\n'):
            flag = True
        else:
            flag = False
    elif line.decode().startswith('///'):
        line_to_write = ec + '\t' + ec_name[ec] + '\n'
        fout.write(line_to_write)
f.close()
tar.close()
fout.close()


# Organism info
org2tax = dict()
fhand = open(keggdir + '/kegg/genes/misc/taxonomic_rank')
for line in fhand:
    if not line.startswith('#'):
        org2tax[line.split('\t')[0]] = line.split('\t')[1]
fhand.close()

fhand = open(keggdir + '/kegg/genes/misc/taxonomy')
fout = open('supplOutput/organism.txt', 'w')
fout.write('Org code' + '\t' + 'Entry' + '\t' + 'Name' + '\t' + 'NCBI Taxonomy' + '\n')
for line in fhand:
    if not line.startswith('#'):
        if line.split('\t')[1] in org2tax:
            tax = org2tax[line.split('\t')[1]]
        else:
            tax = ''
        line_to_write = line.split('\t')[1] + '\t' + line.split('\t')[2] + '\t' + line.split('\t')[3].rstrip() + '\t' + tax + '\n'
        fout.write(line_to_write)
fhand.close()
fout.close()


# Domain info
fout = open('supplOutput/domain.txt', 'w')
fout.write('Abbreviation' + '\t' + 'Full name' + '\n')
line_to_write = 'A\tArchaea\n' + 'B\tBacteria\n' + 'E\tEukaryotes\n'
fout.write(line_to_write)
fout.close()


# Gene info
def saveProtID(org):
    tar = tarfile.open(keggdir + '/kegg/genes/organisms/' + org + '/' + org + '_link.tar.gz', 'r:gz')
    f_ncbiPid = tar.extractfile(org + '_ncbi-proteinid.list')
    info = pd.read_table(f_ncbiPid, sep='\t', header=None)
    f_ncbiPid.close()
    tar.close()
    df = pd.concat([info[0].str.replace(org + ':',''),info[1].str.replace('ncbi-proteinid:','')], axis=1)
    df.columns = ['KEGG gene id','NCBI protein id']
    df.to_csv('supplOutput/gene/' + org +  '.txt', sep='\t', index=False)

exceptionlist = list()
for root, dirs, files in os.walk(keggdir + '/kegg/genes/organisms/'):
    for org in dirs:
        try:
            saveProtID(org)
        except:
            exceptionlist.append(org)
print(len(exceptionlist),'organisms', exceptionlist, 'do not have supplementary gene file due to the missing file of ncbi-proteinid.list')

time_end = time.time()
print('time cost:',round(time_end-time_start),'s')

[16:27:37] WARNING: not removing hydrogen atom without neighbors
[16:27:39] WARNING: not removing hydrogen atom with dummy atom neighbors
[16:27:41] Unrecognized radical value 0 for atom 0 on line 31

[16:27:42] WARNING: not removing hydrogen atom with dummy atom neighbors
[16:27:45] Explicit valence for atom # 25 N, 4, is greater than permitted
[16:28:25] Explicit valence for atom # 5 N, 4, is greater than permitted
[16:28:26] Explicit valence for atom # 11 B, 7, is greater than permitted
[16:28:27] Explicit valence for atom # 5 O, 4, is greater than permitted
[16:28:43] Explicit valence for atom # 1 Cl, 4, is greater than permitted
[16:28:43] Explicit valence for atom # 5 O, 3, is greater than permitted
[16:28:43] Explicit valence for atom # 3 O, 3, is greater than permitted
[16:28:44] Warning: conflicting stereochemistry at atom 19 ignored. by rule 1a.
[16:28:44] Warning: conflicting stereochemistry at atom 19 ignored. by rule 1a.
[16:28:46] Explicit valence for atom # 0 Si, 8, is g

6 organisms ['pfd', 'lja', 'sco', 'smin', 'pfh', 'dosa'] do not have supplementary gene file due to the missing file of ncbi-proteinid.list
time cost: 418 s
